In [1]:
import pandas as pd
import numpy as np
from tqdm import trange,tqdm
from sklearn.model_selection import train_test_split

import re,os
import random

In [300]:
sample_num = 20000

# patent_dic.txt & label.txt

## 2012 patent

In [301]:
patent_raw = pd.read_excel("E:/Pythonworkspace/patent/process_data/sample"+str(sample_num)+"/sample.xlsx", encoding='utf-8')

In [302]:
len(patent_raw)

20000

In [303]:
df_patent_raw = pd.DataFrame({'application_id':patent_raw['application_id'],'result':patent_raw['result']})

## 2012 parent

In [304]:
patent_parent = pd.read_csv("E:/Pythonworkspace/patent/process_data/sample"+str(sample_num)+"/network/parent.csv", encoding='utf-8')

In [305]:
parent_result = [0 for i in range(len(patent_parent))]
df_patent_parent = pd.DataFrame({'application_id':patent_parent['parent_id'],'result':parent_result})

In [306]:
df_patent_parent = df_patent_parent.drop_duplicates()
len(df_patent_parent)

7971

## dynamic years

In [307]:
start_year = "2011"
end_year = "2011"

path_file = "E:/Pythonworkspace/patent/process_data/sample" + str(sample_num) + "/network/"
network_path = path_file + start_year + "-" + end_year

### inventor before

In [308]:
inventor_patent_file = network_path + "/I-P-label.csv"
df_inventor_patent = pd.read_csv(inventor_patent_file, encoding='utf-8')
len(df_inventor_patent)

30872

### assignee before

In [309]:
assignee_patent_file = network_path + "/A-P-label.csv"
df_assignee_patent = pd.read_csv(assignee_patent_file, encoding='utf-8')
len(df_assignee_patent)

125017

## concat and drop_duplicates

In [310]:
df_1 = pd.concat([df_patent_raw,df_patent_parent], axis=0)
df_1 = df_1.drop_duplicates(subset=['application_id'], keep='first')
df_2 = pd.concat([df_inventor_patent,df_assignee_patent], axis=0)
df_2 = df_2.drop_duplicates(subset=['application_id'], keep='first')
df = pd.concat([df_1,df_2], axis=0)
df = df.drop_duplicates(subset=['application_id'], keep='first')
len(df)

166201

## patent_dic.csv

In [311]:
index_raw = [i for i in range(len(df))]
patent_dic = pd.DataFrame({'index':index_raw,'application_id':df['application_id']})

In [833]:
patent_dic_file = network_path + "/patent_dic.csv"
patent_dic.to_csv(patent_dic_file, mode='a', header=0, index=False, sep=',')

## patent_label.xlsx

In [834]:
patent_label = pd.DataFrame({'index':index_raw,'result':df['result']})

In [835]:
patent_label_file = network_path + "/patent_label.csv"
patent_label.to_csv(patent_label_file, mode='a', header=0, index=False, sep=',')

# inventor_dic.txt

In [312]:
inventor_file = path_file + "PI+.xlsx"
inventor = pd.read_excel(inventor_file, encoding='utf-8')

In [313]:
inventor_name = inventor['name'].tolist()
inventor_list = set(inventor_name)
inventor_list = list(inventor_list)
len(inventor_list)

50112

In [314]:
inventor_index = [i for i in range(len(df), len(df)+len(inventor_list))]
df_inventor = pd.DataFrame({'index':inventor_index,'application_id':inventor_list})

In [842]:
df_inventor.to_csv(network_path + "/inventor_dic.csv", mode='a', header=0, index=False, sep=',')

# assignee.txt

In [315]:
assignee_file = path_file + "PA.xlsx"
assignee = pd.read_excel(assignee_file, encoding='utf-8')

In [316]:
assignee_name = assignee['name'].tolist()
assignee_list = set(assignee_name)
assignee_list = list(assignee_list)
len(assignee_list)

6611

In [317]:
assignee_index = [i for i in range(len(df)+len(inventor_list), len(df)+len(inventor_list)+len(assignee_list))]
df_assignee = pd.DataFrame({'index':assignee_index,'application_id':assignee_list})

In [849]:
df_assignee.to_csv(network_path + "/assignee_dic.csv", mode='a', header=0, index=False, sep=',')

# 建立字典

## 建立assignee字典

In [318]:
assigne_dic = {}

for i in tqdm(range(len(df_assignee)), ncols=60):
    if not df_assignee['application_id'][i] in assigne_dic:
        assigne_dic[df_assignee['application_id'][i]] = df_assignee['index'][i]

100%|████████████████| 6611/6611 [00:00<00:00, 14763.23it/s]


## 建立patent字典

In [320]:
patent_index_dic = {}

for i in tqdm(range(len(patent_dic)), ncols=60):
    if not patent_dic['application_id'].iloc[i] in patent_index_dic:
        patent_index_dic[str(patent_dic['application_id'].iloc[i])] = patent_dic['index'].iloc[i]

100%|████████████| 166201/166201 [00:10<00:00, 16205.88it/s]


## 建立inventor字典

In [319]:
inventor_dic = {}

for i in tqdm(range(len(df_inventor)), ncols=60):
    if not df_inventor['application_id'][i] in inventor_dic:
        inventor_dic[df_inventor['application_id'][i]] = df_inventor['index'][i]

100%|██████████████| 50112/50112 [00:02<00:00, 17314.29it/s]


# PP.txt

In [321]:
parent = pd.read_csv(path_file + "parent.csv", encoding='utf-8')

In [322]:
application_id_index_list = []
parent_id_index_list = []

for i in trange(len(parent), ncols=50):
    application_id = parent['application_id'][i]
    application_id_index_list.append(patent_index_dic[str(application_id)])
    parent_id = parent['parent_id'][i]
    parent_id_index_list.append(patent_index_dic[str(parent_id)])

100%|██████| 8770/8770 [00:00<00:00, 24840.27it/s]


In [323]:
len(application_id_index_list)

8770

In [46]:
parent_patent = pd.DataFrame( {'application_id': application_id_index_list,'parent_id': parent_id_index_list})
parent_patent.to_csv(network_path + "/PP.csv", mode='a', header=0, index=False, sep=',')

# PA.txt

## PA

In [324]:
PA = assignee
application_id_index_list = []
name_index_list = []

for i in trange(len(PA), ncols=50):
    application_id = PA['application_id'][i]
    application_id_index_list.append(patent_index_dic[str(application_id)])
    name = PA['name'][i]
    name_index_list.append(assigne_dic[name])

100%|████| 14005/14005 [00:00<00:00, 23404.22it/s]


In [325]:
len(application_id_index_list)

14005

In [867]:
new_PA = pd.DataFrame( {'application_id': application_id_index_list,'name': name_index_list} )
new_PA.to_csv(network_path + "/PA.csv", mode='a', header=0, index=False, sep=',')

## A-GP

In [326]:
A_GP_file = network_path + "/A-P-"+ start_year + "-" + end_year +".xlsx"
A_GP = pd.read_excel(A_GP_file, encoding='utf-8')

In [327]:
application_id_index_list = []
name_index_list = []

for i in trange(len(A_GP), ncols=50):
    application_id = A_GP['application_id'][i]
    application_id_index_list.append(patent_index_dic[str(application_id)])
    name = A_GP['name'][i]
    name_index_list.append(assigne_dic[name])

100%|██| 125032/125032 [00:04<00:00, 25621.61it/s]


In [328]:
len(application_id_index_list)

125032

In [871]:
new_A_GP = pd.DataFrame( {'application_id': application_id_index_list,'name': name_index_list} )
new_A_GP.to_csv(network_path + "/PA.csv", mode='a', header=0, index=False, sep=',')

# PI.txt

## PI

In [329]:
PI = inventor
application_id_index_list = []
name_index_list = []

for i in trange(len(PI), ncols=50):
    application_id = PI['application_id'][i]
    application_id_index_list.append(patent_index_dic[str(application_id)])
    name = PI['name'][i]
    name_index_list.append(inventor_dic[name])

100%|████| 53824/53824 [00:02<00:00, 25360.90it/s]


In [330]:
len(application_id_index_list)

53824

In [875]:
new_PI = pd.DataFrame( {'application_id': application_id_index_list,'name': name_index_list} )
new_PI.to_csv(network_path + "/PI.csv", mode='a', header=0, index=False, sep=',')

## I-GP

In [331]:
I_GP_file = network_path + "/I-P-"+ start_year + "-" + end_year +".xlsx"
I_GP = pd.read_excel(I_GP_file, encoding='utf-8')

In [332]:
application_id_index_list = []
name_index_list = []

for i in trange(len(I_GP), ncols=50):
    application_id = I_GP['application_id'][i]
    application_id_index_list.append(patent_index_dic[str(application_id)])
    name = I_GP['name'][i]
    name_index_list.append(inventor_dic[name])

100%|████| 53019/53019 [00:02<00:00, 25906.87it/s]


In [333]:
len(application_id_index_list)

53019

In [879]:
new_I_GP = pd.DataFrame( {'application_id': application_id_index_list,'name': name_index_list} )
new_I_GP.to_csv(network_path + "/PI.csv", mode='a', header=0, index=False, sep=',')

# train&test

In [880]:
patent_label.head(2)

,index,result
3887,0,0
1795,1,1


In [881]:
len(patent_label)

166201

In [882]:
# 划分训练集
for i in tqdm(range(10), ncols=60):
    X_train, X_test, y_train, y_test = train_test_split(patent_label['index'].values.tolist(), patent_label['result'].values.tolist(), test_size=0.5,random_state=i)
    
    p = re.compile(r',|\[|\]', re.S)
    X_train_text = p.sub('', str(X_train))
    X_test_text = p.sub('', str(X_test))
    
    with open(network_path + "/paper_label_train_idx.txt", 'a') as f:
        f.write(X_train_text)
        f.write('\n')
    with open(network_path + "/paper_label_test_idx.txt", 'a') as f:
        f.write(X_test_text)
        f.write('\n')


100%|███████████████████████| 10/10 [00:02<00:00,  4.22it/s]


# PP补充citations

## for prior patent

### appID_to_patentID

In [60]:
sum_2011 = pd.read_excel("E:/Pythonworkspace/patent/patent_data/GrantedSum/2011.xlsx", encoding='utf-8')
sum_2010 = pd.read_excel("E:/Pythonworkspace/patent/patent_data/GrantedSum/2010.xlsx", encoding='utf-8')
sum_2009 = pd.read_excel("E:/Pythonworkspace/patent/patent_data/GrantedSum/2009.xlsx", encoding='utf-8')
sum_2008 = pd.read_excel("E:/Pythonworkspace/patent/patent_data/GrantedSum/2008.xlsx", encoding='utf-8')
sum_2007 = pd.read_excel("E:/Pythonworkspace/patent/patent_data/GrantedSum/2007.xlsx", encoding='utf-8')
sum_2006 = pd.read_excel("E:/Pythonworkspace/patent/patent_data/GrantedSum/2006.xlsx", encoding='utf-8')
sum_2005 = pd.read_excel("E:/Pythonworkspace/patent/patent_data/GrantedSum/2005.xlsx", encoding='utf-8')
sum_2004 = pd.read_excel("E:/Pythonworkspace/patent/patent_data/GrantedSum/2004.xlsx", encoding='utf-8')
sum_2003 = pd.read_excel("E:/Pythonworkspace/patent/patent_data/GrantedSum/2003.xlsx", encoding='utf-8')
sum_2002 = pd.read_excel("E:/Pythonworkspace/patent/patent_data/GrantedSum/2002.xlsx", encoding='utf-8')
sum_2001 = pd.read_excel("E:/Pythonworkspace/patent/patent_data/GrantedSum/2001.xlsx", encoding='utf-8')

In [66]:
patent_label_file = network_path + "/patent_dic.csv"
patent_label = pd.read_csv(patent_label_file, encoding='utf-8', header=None)
len(patent_label)

109283

In [72]:
prior_patent = patent_label.iloc[sample_num:]
len(prior_patent)

104283

In [102]:
citations_file = network_path + "/appID_to_patentID.csv"
for index in tqdm(range(len(prior_patent)), ncols=50):
    try:
        application_id = int(prior_patent[0].iloc[index])
        for year in range(2001, 2012):
            if year == 2001:
                patent_id_year = sum_2001[sum_2001['application_id']==application_id]
            if year == 2002:
                patent_id_year = sum_2001[sum_2001['application_id']==application_id]
            if year == 2003:
                patent_id_year = sum_2001[sum_2001['application_id']==application_id]
            if year == 2004:
                patent_id_year = sum_2001[sum_2001['application_id']==application_id]
            if year == 2005:
                patent_id_year = sum_2001[sum_2001['application_id']==application_id]
            if year == 2006:
                patent_id_year = sum_2006[sum_2006['application_id']==application_id]
            if year == 2007:
                patent_id_year = sum_2007[sum_2007['application_id']==application_id]
            if year == 2008:
                patent_id_year = sum_2008[sum_2008['application_id']==application_id]
            if year == 2009:
                patent_id_year = sum_2009[sum_2009['application_id']==application_id]
            if year == 2010:
                patent_id_year = sum_2010[sum_2010['application_id']==application_id]
            if year == 2011:
                patent_id_year = sum_2011[sum_2011['application_id']==application_id]
            if len(patent_id_year)>0:
                patent_id = patent_id_year['patent_id'].values[0]            

                df = pd.DataFrame({'application_id':[application_id],'patent_id':[patent_id]})
                if os.path.exists(citations_file):
                    df.to_csv(citations_file, header=0, mode='a', index=False, sep=',')
                else:
                    df.to_csv(citations_file, mode='a', index=False, sep=',')

                break
    except:
        pass

100%|█████| 104283/104283 [18:21<00:00, 94.64it/s]


### get citations

In [74]:
citations_file = network_path + "/appID_to_patentID.csv"
appID_to_patentID = pd.read_csv(citations_file, encoding='utf-8')

In [75]:
patent_id_list = appID_to_patentID['patent_id'].values.tolist()
patent_id_list = set(patent_id_list)

In [76]:
year_list = ["2019", "2018", "2017", "2016", "2015", "2014", "2013", "2012", "2011", "2010", 
             "2009", "2008", "2007", "2006", "2005", "2004", "2003", "2002", "2001"]

In [77]:
save_file = network_path + "/citation.csv"
for year in year_list:
    citations_by_year = pd.read_csv("E:/Pythonworkspace/patent/process_data/citations/"+year+".csv", encoding='utf-8')
    for index in tqdm(range(len(citations_by_year)), ncols=60):    
        patent_id_1 = citations_by_year.iloc[index]['citation_id']
        patent_id_2 = citations_by_year.iloc[index]['patent_id']
        if patent_id_1 in patent_id_list:
            if patent_id_2 in patent_id_list:
                patent_id_1 = appID_to_patentID[appID_to_patentID['patent_id']==patent_id_1]
                application_id_1 = patent_id_1['application_id'].values[0]
                patent_id_2 = appID_to_patentID[appID_to_patentID['patent_id']==patent_id_2]
                application_id_2 = patent_id_2['application_id'].values[0]

                df = pd.DataFrame({'application_id_1':[application_id_1],'application_id_2':[application_id_2]})
                if os.path.exists(save_file):
                    df.to_csv(save_file, header=0, mode='a', index=False, sep=',')
                else:
                    df.to_csv(save_file, mode='a', index=False, sep=',')

100%|███████████| 4627298/4627298 [24:23<00:00, 3161.21it/s]


### 将appID转成index

In [334]:
citation = pd.read_csv(network_path + "/citation.csv", encoding='utf-8')

count = 0
for index in tqdm(range(len(citation)), ncols=50):
    application_id_1 = citation.iloc[index]['application_id_1']
    application_id_1 = str(application_id_1)
    application_id_2 = citation.iloc[index]['application_id_2']
    application_id_2 = str(application_id_2)
    
    if application_id_1 in patent_index_dic.keys():
        if application_id_2 in patent_index_dic.keys():
            count = count + 1
            application_index_1 = patent_index_dic[application_id_1]
            application_index_2 = patent_index_dic[application_id_2]
            
            citations_patent = pd.DataFrame( {'application_id': [application_index_1],'parent_id': [application_index_2]})
            citations_patent.to_csv(network_path + "/PP.csv", mode='a', header=0, index=False, sep=',')

100%|████████| 9452/9452 [00:16<00:00, 588.97it/s]


In [335]:
count

9452

## for application

### appID_to_location

In [218]:
sample_file = "E:/Pythonworkspace/patent/process_data/sample" + str(sample_num) + "/sample.xlsx"
sample = pd.read_excel(sample_file, encoding='utf-8')

In [219]:
application_id_list = sample['application_id'].values.tolist()
application_id_list = set(application_id_list)

In [220]:
citations_by_year = pd.read_csv("E:/Pythonworkspace/patent/process_data/citations_app/2012_appID.csv", encoding='utf-8')

In [221]:
len(citations_by_year)

177895

In [222]:
filepath = path_file + "citations_app.csv"
for index in tqdm(range(len(citations_by_year)), ncols=70):
    application_id = citations_by_year['application_id'].iloc[index]
    if application_id in application_id_list:
        patent_id = citations_by_year['patent_id'].iloc[index]
        dataframe = pd.DataFrame({'application_id':[application_id],'patent_id':[patent_id]})
        if os.path.exists(filepath):
            dataframe.to_csv(filepath, header=0, mode='a', index=False, sep=',')
        else:
            dataframe.to_csv(filepath, mode='a', index=False, sep=',')

100%|███████████████████████| 177895/177895 [00:31<00:00, 5695.30it/s]


### 将patent_id对应到application_id

In [336]:
citations_file = network_path + "/appID_to_patentID.csv"
appID_to_patentID = pd.read_csv(citations_file, encoding='utf-8')
sample_citation = pd.read_csv(path_file + "citations_app.csv", encoding='utf-8')

In [337]:
application_id_list = []
cited_id_list = []

for index in tqdm(range(len(sample_citation)), ncols=50):
    application_id = sample_citation['application_id'].iloc[index]
    patent_id = sample_citation['patent_id'].iloc[index]
    if len(appID_to_patentID[appID_to_patentID['patent_id']==patent_id]) > 0:
        cited_id = appID_to_patentID[appID_to_patentID['patent_id']==patent_id]['application_id'].values
        application_id_list.append(application_id)
        cited_id_list.append(cited_id)

100%|███████| 10838/10838 [01:49<00:00, 98.74it/s]


### 将appl_id转为index

In [338]:
count = 0
for i in tqdm(range(len(application_id_list)), ncols=50):
    application_id = str(application_id_list[i])
    cited_id = str(cited_id_list[i][0])
    
    if cited_id in patent_index_dic.keys():
        cited_index = patent_index_dic[cited_id]
        if application_id in patent_index_dic.keys():
            count = count + 1
            application_index = patent_index_dic[application_id]
            
            citations_patent = pd.DataFrame( {'application_id': [application_index],'parent_id': [cited_index]})
            citations_patent.to_csv(network_path + "/PP.csv", mode='a', header=0, index=False, sep=',')

100%|██████████| 154/154 [00:00<00:00, 622.63it/s]


In [339]:
count

154